In [ ]:
import vpython
import qutip
import random
import math
import numpy as np
import cmath
import mpmath

#

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

def state_polynomial(v):
    polynomial = v.T.tolist()
    return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))]

def polynomial_roots(polynomial):
    try:
        roots = [complex(root) for root in mpmath.polyroots(polynomial)]
    except:
        return [complex(0,0) for i in range(len(polynomial)-1)]
    return roots

def c_xyz(c):
    if c == float('inf'):
        return [0,0,1]
    x = c.real
    y = c.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def state_xyz(state):
    return c_xyz(polynomial_roots(state_polynomial(state))[0])

#

def A(t):
    return np.array([[np.cosh(t), np.sinh(t)], [np.sinh(t), np.cosh(t)]])

def N(t):
    return np.array([[1, 0], [t, 1]])

def K(t):
    return np.array([[np.cos(t), -1*np.sin(t)],[np.sin(t), np.cos(t)]])

#

def mobius_rotate(m, latitude, longitude):
    state = spherical_state(latitude, longitude)
    arrow = np.outer(state, np.conjugate(state.T))
    return np.dot(np.conjugate(arrow.T), np.dot(m, arrow))

def apply_mobius(m, states):
    return [np.dot(m, state) for state in states]

#
    
def spherical_state(theta, phi):
    return np.array([math.cos(theta/2), cmath.exp(complex(0,1)*phi)*theta/2])

def spherical_xyz(theta, phi):
    x = math.sin(theta)*math.cos(phi)
    y = math.sin(theta)*math.sin(phi)
    z = math.cos(theta)
    return [x, y, z]

#

def random_stars(n):
    return [[random.uniform(0, math.pi), random.uniform(0, 2*math.pi)] for i in range(n)]

#

vpython.scene.width = 1000
vpython.scene.height = 1000
vpython.scene.range = 1.5
vpython.scene.forward = vpython.vector(0, 1, 0)
vpython.scene.up = vpython.vector(-1, 0, 0)
vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)
vearth = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.1, color=vpython.color.cyan, opacity=0.5, emissive=True)
vobserver = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.01, color=vpython.color.yellow, opacity=0.5, emissive=True, make_trail=True)
        
vstars = []

#

n = 25
states = []
for i in range(n):
    theta, phi = random.uniform(0, math.pi), random.uniform(0, 2*math.pi)

    state = spherical_state(theta, phi)
    states.append(state)

    x, y, z = spherical_xyz(theta, phi)
    vstars.append(vpython.sphere(pos=vpython.vector(x, y, z),\
                                 radius=0.01, emissive=True, make_trail=True))
    
#
latitude = 0
longitude = 0

flag = False
step = 0.01

def keyboard(event):
    global mobius
    global states
    global flag
    global vstars
    global step
    global latitude
    global longitude
    key = event.key
    if key == "a":
        latitude -= step
    elif key == "d":
        latitude += step
    elif key == "w":
        longitude += step
    elif key == "s":
        longitude -= step
    elif key == "z":
        step -= 0.01
    elif key == "x":
        step += 0.01
    elif key == "f":
        states = apply_mobius(mobius_rotate(A(-1*step), latitude, longitude), states)
    elif key == "h":
        states = apply_mobius(mobius_rotate(A(step), latitude, longitude), states)
    elif key == "t":
        states = apply_mobius(mobius_rotate(N(step), latitude, longitude), states)
    elif key == "g":
        states = apply_mobius(mobius_rotate(N(-1*step), latitude, longitude), states)
    elif key == "v":
        states = apply_mobius(mobius_rotate(K(-1*step), latitude, longitude), states)
    elif key == "b":
        states = apply_mobius(mobius_rotate(K(step), latitude, longitude), states)
    elif key == "y":
        states = []
        new_stars = random_stars(n)
        for i in range(n):
            theta, phi = new_stars[i]
            state = spherical_state(theta, phi)
            states.append(state)
    elif key == "p":
        for vstar in vstars:
            vstar.clear_trail()
    flag = True
        
vpython.scene.bind('keydown', keyboard)

#

while True:
    vpython.rate(20)
    if flag:
        for i in range(n):
            x, y, z = state_xyz(states[i])
            vstars[i].pos = vpython.vector(x, y, z)
        vobserver.pos = vpython.vector(*[0.1*coord for coord in spherical_xyz(latitude, longitude)])
        flag = False

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>